<a href="https://colab.research.google.com/github/nanoNNsk/superaiss5_hackathon/blob/main/nlp_hackathon_superaiss5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# upload file

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# data prepare

In [11]:
!pip install pandas

In [64]:
#function for load data in folder with .txt
import os
import pandas as pd
def load_data(folder_path):
    data = []
    for filename in sorted(os.listdir(folder_path)):
        if filename.endswith(".txt"):
            with open(os.path.join(folder_path, filename), 'r', encoding='utf-8') as file:
                for line_number, line in enumerate(file):
                    file_id = filename.split('.')[0]  # ตัดส่วน .txt ออกไป
                    data.append({'id': f'{file_id}_{line_number}', 'text': line.strip()})
    df = pd.DataFrame(data)
    split_columns = df['text'].str.split('\t', expand=True)
    result = pd.concat([df['id'], split_columns], axis=1)
    result.columns = ['id', 'word', 'pos', 'ner', 'more']

    return result

def load_data_test(folder_path):
    data = []
    for filename in sorted(os.listdir(folder_path)):
        if filename.endswith(".txt"):
            with open(os.path.join(folder_path, filename), 'r', encoding='utf-8') as file:
                for line_number, line in enumerate(file):
                    file_id = filename.split('.')[0]  # ตัดส่วน .txt ออกไป
                    data.append({'id': f'{file_id}_{line_number}', 'text': line.strip()})
    df = pd.DataFrame(data)
    split_columns = df['text'].str.split('\t', expand=True)
    result = pd.concat([df['id'], split_columns], axis=1)
    result.columns = ['id', 'word', 'pos', 'ner']

    return result

In [65]:
# set path to dataset
train_folder = '/content/drive/MyDrive/hackathon_superaiss5/nlp/superai_ner/train/train'
eval_folder = '/content/drive/MyDrive/hackathon_superaiss5/nlp/superai_ner/eval/eval'
test_folder = '/content/drive/MyDrive/hackathon_superaiss5/nlp/superai_ner/test/test'

In [75]:
train_data = load_data(train_folder)
eval_data = load_data(eval_folder)
test_data = load_data_test(test_folder)

In [68]:
train_data.head()

,id,word,pos,ner,more
0,00001_0,สภาสังคมสงเคราะห์แห่งประเทศ,NN,B_ORG,B_CLS
1,00001_1,ไทย,NN,E_ORG,I_CLS
2,00001_2,จี้,VV,O,I_CLS
3,00001_3,ศาล,NN,O,I_CLS
4,00001_4,ไฟเขียว,VV,O,I_CLS


In [69]:
test_data.head()

,id,word,pos,ner
0,03795_0,รัฐ,NN,B_CLS
1,03795_1,ถังแตก,VV,I_CLS
2,03795_2,วิก,NN,I_CLS
3,03795_3,_,NN,I_CLS
4,03795_4,7,NN,I_CLS


In [76]:
train_data.drop(columns=['pos','more'], inplace=True)
eval_data.drop(columns=['pos','more'], inplace=True)
test_data.drop(columns=['pos','ner'], inplace=True)

In [77]:
train_data.head()

,id,word,ner
0,00001_0,สภาสังคมสงเคราะห์แห่งประเทศ,B_ORG
1,00001_1,ไทย,E_ORG
2,00001_2,จี้,O
3,00001_3,ศาล,O
4,00001_4,ไฟเขียว,O


In [78]:
eval_data.head()

,id,word,ner
0,04278_0,โฆษก,O
1,04278_1,กอส.,B_ORG
2,04278_2,ตำหนิ,O
3,04278_3,แมนฯ,B_ORG
4,04278_4,_,I_ORG


In [79]:
test_data.head()

,id,word
0,03795_0,รัฐ
1,03795_1,ถังแตก
2,03795_2,วิก
3,03795_3,_
4,03795_4,7


In [38]:
labels = pd.read_csv('/content/drive/MyDrive/hackathon_superaiss5/nlp/superai_ner/tag_list.csv')
labels.head()

,tag,class
0,O,0
1,B_ORG,1
2,B_PER,2
3,B_LOC,3
4,B_MEA,4


# load model